In [4]:
# 107303538_1209_hw12_1

In [5]:
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim # 最佳化資料庫
from torchvision import datasets
from matplotlib import pyplot as plt
from torchvision import transforms

datasets  為tensor和transform

In [6]:
# 訓練集
import os
data_path = os.getcwd() # 在datasets有內建

transformed_cifar10 = datasets.CIFAR10(
  data_path, train=True, download=True,
  transform=transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize((0.4914, 0.4822, 0.4465),
  (0.2470, 0.2435, 0.2616))
]))

# val組
transformed_cifar10_val = datasets.CIFAR10(
  data_path, train=False, download=True,
  transform=transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize((0.4914, 0.4822, 0.4465),
  (0.2470, 0.2435, 0.2616))
]))

Files already downloaded and verified
Files already downloaded and verified


分辨飛機與鳥cifar2

In [7]:
# 只取鳥和飛機
label_map = {0: 0, 2: 1} # {飛機: 新的array的0位置, 鳥: 新的array的1位置} ex.為0就轉換為0 為2就轉換為1
class_names = ['airplane', 'bird'] 

# 若label為0和2 匯出其img和label label放到label_map轉換整數值
cifar2 = [(img, label_map[label]) 
      for img, label in transformed_cifar10
      if label in [0, 2]]
cifar2_val = [(img, label_map[label])
        for img, label in transformed_cifar10_val
        if label in [0, 2]]

DataLoader

In [16]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64, shuffle=True) # DataLaoder去批次(一次64個數據)訓練
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64, shuffle=False) # 不用洗牌

訓練模型(LogSoftmax)

In [17]:
model = nn.Sequential(
      nn.Linear(3072, 1024),
      nn.Tanh(),
      nn.Linear(1024, 512),
      nn.Tanh(),
      nn.Linear(512, 128),
      nn.Tanh(),
      nn.Linear(128, 2),
      nn.LogSoftmax(dim=1))

In [18]:
loss_fn = nn.NLLLoss() # 其相對應的loss function

In [19]:
learning_rate = 1e-2 # 學習率
optimizer = optim.SGD(model.parameters(), lr=learning_rate) # 優化器

In [34]:
n_epochs = 1
for epoch in range(n_epochs):
  for imgs, labels in train_loader: # 已經只有鳥和飛機的資料庫 且設定批次訓練64個 train_loader
    batch_size = imgs.shape[0] # 一次訓練64個 train_loader為BCHW

    outputs = model(imgs.view(batch_size, -1)) # [64, 3072]
    # nn訓練模型標準化輸入為(batch, 圖片其他參數)

    loss = loss_fn(outputs, labels) # train_loader的labels已經是tensor了
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 0.424032


正確率

In [47]:
correct = 0 
total = 0

with torch.no_grad(): # 不用回傳
  for imgs, labels in val_loader: # 驗證集
    batch_size = imgs.shape[0]

    outputs = model(imgs.view(batch_size, -1)) # 驗證集訓練結果

    _, predicted = torch.max(outputs, dim=1) # 回傳 (最大值的數字, 最大值的label) 在此為驗證的label

    total += labels.shape[0] # 每次訓練幾筆batch 都疊加 得到總共訓練的筆數
    correct += int((predicted == labels).sum()) # 若預測的結果和labels相似 則將tensor.bool值相加(true=1/false=0) 得到總共的正確筆數ex.tensor(48) 轉int
    
print("Accuracy: %f", correct / total)

Accuracy: %f 0.7965


去看主要是哪層hidden_layer貢獻w和b

In [48]:
numel_list = [p.numel() # 回傳數量
        for p in model.parameters() 
        if p.requires_grad == True] # 有貢獻到回傳的
# Parameter有兩個參數(data=None, requires_grad=True)
sum(numel_list), numel_list # 顯示1024貢獻了大多數的訓練 所以可以只用一層即可(效率更好)

(3737474, [3145728, 1024, 524288, 512, 65536, 128, 256, 2])